In [3]:
import requests
import urllib
from bs4 import BeautifulSoup

In [4]:
blocklist = [
    "Main_Page",
    "Help:",
    "Special:",
    "Portal:",
    "Talk:",
    "Wikipedia:",
    "Template:"
]

In [5]:
urllib.parse.unquote("/wiki/Hamilton%E2%80%93Jacobi%E2%80%93Bellman_equation")

'/wiki/Hamilton–Jacobi–Bellman_equation'

In [136]:
def parse_wiki_link(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')


# dont look if wrapper class is cite
    wiki_content_links = set()
    for link in soup.find_all('a', href=True):
        if link["href"] == "#cite_ref-1":
            break
        clean_link = link["href"]
        clean_link = urllib.parse.unquote(clean_link)
        if clean_link.startswith("/wiki/"):
            wiki_link = clean_link[6:]
            if any(x in wiki_link for x in blocklist):
                continue
            wiki_content_links.add(clean_link)
    return wiki_content_links

In [137]:
link = "https://en.wikipedia.org/wiki/Hamilton–Jacobi–Bellman_equation"
l = parse_wiki_link(link)

In [138]:
print(l)

{'/wiki/Value_function', '/wiki/Elliptic_partial_differential_equation', '/wiki/Taylor_expansion', '/wiki/Rudolf_E._Kálmán', '/wiki/Sum-of-squares_optimization', '/wiki/Michael_G._Crandall', '/wiki/Artificial_neural_network', '/wiki/Control_(optimal_control_theory)', '/wiki/Classical_physics', '/wiki/Bequest_value', '/wiki/Itô_calculus', '/wiki/Minimax_solution', '/wiki/Partial_differential_equation', "/wiki/Pontryagin's_maximum_principle", '/wiki/Discrete_time_and_continuous_time', '/wiki/Linear-quadratic-Gaussian_control', '/wiki/Necessary_and_sufficient_condition', '/wiki/Richard_Bellman', '/wiki/Hamilton–Jacobi_equation', '/wiki/Riccati_equation', '/wiki/Difference_equation', '/wiki/Infinitesimal_generator_(stochastic_processes)', '/wiki/Hamiltonian_(control_theory)', '/wiki/Smoothness', '/wiki/Variational_problem', '/wiki/Optimal_control_theory', '/wiki/Brachistochrone_problem', '/wiki/Subderivative', '/wiki/Pierre-Louis_Lions', '/wiki/Backward_induction', '/wiki/Bellman_equation'

In [8]:
class UserHistory:
    def __init__(self, user_history):
        self.user_history = user_history
        # user_vists is a list of links in chronological order ascending
        # user_vists[-1] is the current page
        self.outgoing_links = set() # TODO change this to dict() : key -> count
        #self.ingoing_links = set()
        for link in user_history:
            self.outgoing_links.update(parse_wiki_link(link))
            #self.ingoing_links.update(parse_wiki_ingoing(link))
        already_visited_pages = {link.split("wikipedia.org/"[1]) for link in user_history}
        self.current_link = user_history[-1]

        # remove self-loops
        self.outgoing_links -= already_visited_pages # for k in already_visited_pages: remove
        #self.ingoing_links -= already_visited_pages
        
        # self.outgoing_text


In [88]:
def score_link_similarity(user_history, target):
    # user_history  
    #   incorporate idf (just hyperlinks) -> scrape target/what_links_here (expensive)
    #   or sample 10000 pages and count link frequency and store it somewhere else                       
    #   incorporate ingoing recommendations
    # return score(target | user_history)

    # how many times does target appear in self.outgoing_links
    pass

In [129]:
def link_clean(target):
    title = target.lower()
    title = title[6:]
    title = title.replace("_", " ")
    return title

In [133]:
def score_link_text_similarity(user_history, target):
    # user_history  
    #   incorporate idf (text)
    #   incorporate ingoing recommendations
    # return score(target | user_history)

    # references in div class = reflist
    
    target = link_clean(target)
    r = requests.get(user_history[-1])
    soup = BeautifulSoup(r.content, 'html.parser')
    words = soup.find_all('p')
    total = 0
    for i in words:
        i = str(i)
        i = i.lower()
        total += i.count(target)
    return total

In [134]:
score_link_text_similarity([link], '/wiki/ISBN_(identifier)')


0

In [ ]:
def score_coupling_similarity(user_history, target):
    # user_history  
    #   need to download target and scrape it's links
    # pages are similar if their outgoing (ingoing) links have overlap
    pass

In [91]:
def compute_outgoing_scores_baseline(user_history):
    # composite score_link_similarity and score_link_text_similarity
    # (todo: this filters scores, will do re-ranking with coupling similarity, re-ranking with deeper searches, etc)
    weight = 1.0 # to be tuned
    outgoing_scores = dict()
    for link in user_history.outgoing_links:
        outgoing_scores[link] = score_link_similarity(user_history, link) + weight * score_link_text_similarity(user_history, link)
    return outgoing_scores

In [92]:
def rerank_with_coupling(user_history, baseline_scores, num_rerank):
    sorted_scores = sort(baseline_scores, 'descending')

    for target, score in sorted_scores[num_rerank]:
        coupling_score = score_coupling_similarity(user_history, target)
        

IndentationError: expected an indented block (<ipython-input-92-8676a6193530>, line 1)

In [ ]:
def recommend(ranks):
    pass

In [ ]:
def visualize_history(user_history):
    pass